In [1]:
import numpy as np
import pandas as pd

import lightgbm as lgb

import gc
from datetime import datetime

from data import downcast, cat_to_int

In [4]:
DATADIR = '../kaggle/input/m5-forecasting-accuracy/'

In [5]:
start = datetime.now()

sales = pd.read_csv(f'{DATADIR}sales_train_validation.csv')
calendar = pd.read_csv(f'{DATADIR}calendar.csv')
prices = pd.read_csv(f'{DATADIR}sell_prices.csv')

print('Time:', datetime.now() - start)

Time: 0:00:06.640027


In [6]:
start = datetime.now()

sales = downcast(sales, verbose=True)
calendar = downcast(calendar, verbose=True)
prices = downcast(prices, verbose=True)

print('Time:', datetime.now() - start)

Mem. usage decreased to 95.42 Mb (78.6% reduction)
Mem. usage decreased to  0.14 Mb (33.2% reduction)
Mem. usage decreased to 45.77 Mb (78.1% reduction)
Time: 0:04:43.818737


In [12]:
start = datetime.now()

sales = cat_to_int(sales, ignore=['id'])
calendar = cat_to_int(calendar, ignore=['d'])
prices = cat_to_int(prices)

print('Time:', datetime.now() - start)

Time: 0:00:00.369820


In [30]:
start = datetime.now()

id_cols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
df = pd.melt(sales,
             id_vars = id_cols,
             var_name = 'd',
             value_name = 'sales')

df = df.merge(calendar, on = "d", copy = False)
df = df.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)

print('Time:', datetime.now() - start)

Time: 0:02:08.488369


In [35]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_008_CA_1_validation,1444,3,0,1,0,d_1,12,2011-01-29,11101,...,1,2011,0,0,0,0,0,0,0,0.459961
1,HOBBIES_1_008_CA_1_validation,1444,3,0,1,0,d_2,15,2011-01-30,11101,...,1,2011,0,0,0,0,0,0,0,0.459961
2,HOBBIES_1_008_CA_1_validation,1444,3,0,1,0,d_3,0,2011-01-31,11101,...,1,2011,0,0,0,0,0,0,0,0.459961
3,HOBBIES_1_008_CA_1_validation,1444,3,0,1,0,d_4,0,2011-02-01,11101,...,2,2011,0,0,0,0,1,1,0,0.459961
4,HOBBIES_1_008_CA_1_validation,1444,3,0,1,0,d_5,0,2011-02-02,11101,...,2,2011,0,0,0,0,1,0,1,0.459961
